In [4]:
!pip uninstall tensorflow

^C


In [5]:
!pip install tensorflow[and-cuda]

^C
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached nvidia_cublas_cu12-12.5.3.2-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.5.82-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached nvidia_cuda_nvcc_cu12-12.5.82-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.5.82-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.5.82-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_cudnn_cu12-9.3.0.75-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached nvidia_cufft_cu12-11.2.3.61-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.6.82-py3-none-win_amd64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.3.83-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.5.1.3-py3-none-win_amd64.whl.metadata (1.6 kB)
INFO: pip is looking at multiple versions of tensorflow[and-cuda] to 

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\users\\aaron\\appdata\\local\\programs\\python\\python311\\lib\\site-packages\\google\\_upb\\_message.pyd'
Consider using the `--user` option or check the permissions.



In [6]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"Physical GPUs: {tf.config.list_physical_devices('GPU')}")
print(f"CUDA built: {tf.test.is_built_with_cuda()}")

TensorFlow version: 2.19.0
Physical GPUs: []
CUDA built: False


In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import gc
from datetime import datetime

# GPU Configuration (from your reference code)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]
        )
        print(f"GPU found and configured: {gpus[0].name}")
        print(f"GPU memory limit set to 4096MB")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")
else:
    print("WARNING: No GPU found - training will be slow on CPU")
    
# Verify GPU is being used
print(f"TensorFlow GPU available: {tf.test.is_gpu_available()}")
print(f"TensorFlow built with CUDA: {tf.test.is_built_with_cuda()}")
if gpus:
    print(f"GPU device name: {tf.config.experimental.get_device_details(gpus[0])}")

# Test GPU with a simple operation
try:
    with tf.device('/GPU:0'):
        test_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        result = tf.reduce_sum(test_tensor)
        print(f"GPU test successful: {result.numpy()}")
except Exception as e:
    print(f"GPU test failed: {e}")
    print("Models will run on CPU - expect slower training")

# Data paths
color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            # Cap ages over 5.5 to 5.5
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            # Skip files with non-numeric age (e.g., "xpx")
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    # Process color images (convert to grayscale)
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            assert img_resized.shape == img_size, f"Image {filename} not resized correctly: {img_resized.shape}"
                            img_normalized = img_resized / 255.0
                            images.append(img_normalized)
                            ages.append(class_idx)
    
    # Process grayscale images
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            assert img_resized.shape == img_size, f"Image {filename} not resized correctly: {img_resized.shape}"
                            img_normalized = img_resized / 255.0
                            images.append(img_normalized)
                            ages.append(class_idx)
    
    images = np.array(images)
    images = np.expand_dims(images, axis=-1)
    ages = np.array(ages)
    
    # Verify final dimensions
    assert images.shape[1:3] == img_size, f"Final image dimensions incorrect: {images.shape}"
    print(f"Images loaded with shape: {images.shape}")
    print(f"Classes: {np.unique(ages)} (0=1.5yr, 1=2.5yr, 2=3.5yr, 3=4.5yr, 4=5.5yr)")
    
    return images, ages

def create_model(base_model_name, input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Convert grayscale to RGB for pretrained models
    rgb_inputs = tf.keras.layers.Concatenate()([inputs, inputs, inputs])
    
    if base_model_name == 'ResNet50':
        base_model = tf.keras.applications.ResNet50(
            weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif base_model_name == 'EfficientNetB0':
        base_model = tf.keras.applications.EfficientNetB0(
            weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif base_model_name == 'VGG16':
        base_model = tf.keras.applications.VGG16(
            weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif base_model_name == 'MobileNetV2':
        base_model = tf.keras.applications.MobileNetV2(
            weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif base_model_name == 'InceptionV3':
        base_model = tf.keras.applications.InceptionV3(
            weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif base_model_name == 'DenseNet121':
        base_model = tf.keras.applications.DenseNet121(
            weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    x = base_model(rgb_inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs, name=base_model_name)
    base_model.trainable = False
    
    return model

def train_model(model, X_train, y_train, X_test, y_test, model_name):
    # Convert to categorical
    y_train_cat = tf.keras.utils.to_categorical(y_train, 5)
    y_test_cat = tf.keras.utils.to_categorical(y_test, 5)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_lr=1e-7)
    ]
    
    model.fit(
        X_train, y_train_cat,
        batch_size=32,
        epochs=50,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=0
    )
    
    # Evaluate
    loss, accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
    
    return model, loss, accuracy

# Create timestamped output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_models_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Models will be saved to: {output_dir}")

# Load data
print("Loading images...")
X, y = load_images(color_path, grayscale_path)
print(f"Loaded {len(X)} images, age range: {y.min():.1f}-{y.max():.1f}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Model families to test
model_families = [
    'ResNet50', 'EfficientNetB0', 'VGG16', 
    'MobileNetV2', 'InceptionV3', 'DenseNet121'
]

results = []
best_accuracy = 0
best_family = None

for model_name in model_families:
    print(f"\nTesting {model_name}...")
    
    try:
        model = create_model(model_name, (224, 224, 1))
        trained_model, loss, accuracy = train_model(
            model, X_train, y_train, X_test, y_test, model_name
        )
        
        # Save model with accuracy in filename
        acc_str = f"{accuracy:.3f}".replace('.', 'p')
        model_filename = f"{model_name}_{acc_str}.h5"
        model_path = os.path.join(output_dir, model_filename)
        trained_model.save(model_path)
        
        results.append({
            'model': model_name,
            'accuracy': accuracy,
            'loss': loss,
            'filename': model_filename,
            'full_path': model_path
        })
        
        print(f"{model_name}: Accuracy={accuracy:.3f}, Loss={loss:.3f}, Saved: {model_filename}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_family = model_name
        
        # Cleanup
        del model, trained_model
        tf.keras.backend.clear_session()
        gc.collect()
        
    except Exception as e:
        print(f"Error with {model_name}: {e}")
        continue

# Find best performing family
print(f"\nBest family: {best_family} (accuracy: {best_accuracy:.3f})")

# Test variations within best family
if best_family:
    print(f"\nTesting variations of {best_family}...")
    
    if best_family == 'ResNet50':
        variations = ['ResNet50', 'ResNet101', 'ResNet152']
    elif best_family == 'EfficientNetB0':
        variations = ['EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2']
    else:
        variations = [best_family]
    
    for var_name in variations:
        if var_name == best_family:
            continue
            
        print(f"\nTesting {var_name}...")
        
        try:
            if var_name == 'ResNet101':
                base_model = tf.keras.applications.ResNet101(
                    weights='imagenet', include_top=False, input_shape=(224, 224, 3))
            elif var_name == 'ResNet152':
                base_model = tf.keras.applications.ResNet152V2(
                    weights='imagenet', include_top=False, input_shape=(224, 224, 3))
            elif var_name == 'EfficientNetB1':
                base_model = tf.keras.applications.EfficientNetB1(
                    weights='imagenet', include_top=False, input_shape=(224, 224, 3))
            elif var_name == 'EfficientNetB2':
                base_model = tf.keras.applications.EfficientNetB2(
                    weights='imagenet', include_top=False, input_shape=(224, 224, 3))
            else:
                continue
            
            inputs = tf.keras.Input(shape=(224, 224, 1))
            rgb_inputs = tf.keras.layers.Concatenate()([inputs, inputs, inputs])
            x = base_model(rgb_inputs)
            x = tf.keras.layers.GlobalAveragePooling2D()(x)
            x = tf.keras.layers.Dense(128, activation='relu')(x)
            x = tf.keras.layers.Dropout(0.5)(x)
            outputs = tf.keras.layers.Dense(1, activation='linear')(x)
            model = tf.keras.Model(inputs, outputs, name=var_name)
            base_model.trainable = False
            
            trained_model, mae, accuracy = train_model(
                model, X_train, y_train, X_test, y_test, var_name
            )
            
            acc_str = f"{accuracy:.3f}".replace('.', 'p')
            model_filename = f"{var_name}_{acc_str}.h5"
            model_path = os.path.join(output_dir, model_filename)
            trained_model.save(model_path)
            
            results.append({
                'model': var_name,
                'accuracy': accuracy,
                'mae': mae,
                'filename': model_filename,
                'full_path': model_path
            })
            
            print(f"{var_name}: Accuracy={accuracy:.3f}, MAE={mae:.3f}, Saved: {model_filename}")
            
            del model, trained_model
            tf.keras.backend.clear_session()
            gc.collect()
            
        except Exception as e:
            print(f"Error with {var_name}: {e}")
            continue

# Final results
print(f"\n{'='*50}")
print("FINAL RESULTS")
print(f"{'='*50}")

results.sort(key=lambda x: x['accuracy'], reverse=True)

for i, result in enumerate(results, 1):
    print(f"{i:2d}. {result['model']:15s} - Accuracy: {result['accuracy']:.3f}, MAE: {result['mae']:.3f}")

if results:
    best_result = results[0]
    print(f"\nBest model: {best_result['model']} with {best_result['accuracy']:.3f} accuracy")
    print(f"Saved as: {best_result['filename']}")

print(f"\nTotal models tested: {len(results)}")
print(f"All models saved in folder: {output_dir}")
print("All models saved with accuracy in filename.")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
TensorFlow GPU available: False
TensorFlow built with CUDA: False
GPU test successful: 10.0
Models will be saved to: deer_age_models_20250730_213538
Loading images...
Images loaded with shape: (465, 224, 224, 1)
Classes: [0 1 2 3 4] (0=1.5yr, 1=2.5yr, 2=3.5yr, 3=4.5yr, 4=5.5yr)
Loaded 465 images, age range: 0.0-4.0
Train: 372, Test: 93

Testing ResNet50...



KeyboardInterrupt


KeyboardInterrupt

